In [ ]:
import requests
from lxml import html
import re
import json 
import pandas as pd
import time
import glob
import numpy as np

In [ ]:
!pwd

/content


In [ ]:
import os
path = "CheckPoints"
try:
    os.mkdir(path) 
except FileExistsError: pass 

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}
it_link = 'https://www.shoprite.co.za/search?q=%3AsearchRelevance%3AbrowseAllStoresFacetOff%3AbrowseAllStoresFacetOff&page='
it_link

'https://www.shoprite.co.za/search?q=%3AsearchRelevance%3AbrowseAllStoresFacetOff%3AbrowseAllStoresFacetOff&page='

In [ ]:
class url_links():
    def main_nodes():
        y , dy = 0 , 20
        n = 9162 #total items
        links = [it_link+'0']
        for i in range(0,n-20,dy):
            y = y + dy 
            pg = int(y/dy)
            link = it_link+f'{pg}'
            links.append(link)
        return links




class item_data():
    def mkpg_tree(pgno=0):
        response = requests.get( it_link+f'{pgno}', headers=headers)
        tree = html.fromstring(response.content)
        string = response.text
        pattern = '"price":{ "value":'
        n = len([match.start() for match in re.finditer(pattern, string)]) 
        return tree, n
    
    def xpath_search(tree,path):
        elements = tree.xpath(path)
        attributes_ = []
        for element in range(len(elements)):
            attributes_ = elements[element].attrib
        return attributes_

    def get_pg_data(pgno):
        pg_data = item_data.mkpg_tree(pgno)
        n       = pg_data[1] + 2
        print("There are {} items in this page.".format(pg_data[1]))
        tree    = pg_data[0]
        name_list , item_id_list , price_list , item_links_list , img_links_list = [] , [] , [] , [] , [] 
        if n-2> 0:
             
            for i in range(2,n):
                path = f'/html/body/main/div[4]/div[1]/div[2]/div/div/div/div[2]/div[{i}]'
                attributes = json.loads(item_data.xpath_search(tree,path)['data-product-ga'])
                name, item_id, price = attributes["name"], attributes["id"], attributes["price"]
                
                path = f'/html/body/main/div[4]/div[1]/div[2]/div/div/div/div[2]/div[{i}]/div/figure/figcaption/div/h3/a'
                item_links = 'https://www.shoprite.co.za' + item_data.xpath_search(tree,path)['href']
                
                path = f'/html/body/main/div[4]/div[1]/div[2]/div/div/div/div[2]/div[{i}]/div/figure/div/a/img'
                img_links = 'https://www.shoprite.co.za' + item_data.xpath_search(tree,path)['data-original-src']
                
                
                name_list.append(name)
                item_id_list.append(item_id)
                price_list.append(float(price))
                item_links_list.append(item_links)
                img_links_list.append(img_links)
               
            
        return (name_list, item_id_list, price_list, item_links_list, img_links_list)

    def get_dataframe(pgno):
        itemz = pd.DataFrame(item_data.get_pg_data(pgno)).T
        return itemz

    def all_items(exceptions=[79,]):
        itemz = item_data.get_dataframe(pgno=0)
        stop = False
        if len(item_data.get_pg_data(pgno=2)[0])== 0:
            stop == True
        
        "This limits iterations"
        x, dx = 0 , 1
        while stop == False:
            x = x + dx
            "This limits iterations"
            if x not in exceptions:
                print(x)
                "This limits iterations"
                if len(item_data.get_pg_data(pgno=x)[0]) == 0 or x ==100000:
                    stop = True
                
                current_itemzpg = item_data.get_dataframe(pgno=x)
                itemz = itemz.append(current_itemzpg)
                itemz_cp = itemz.rename(columns={0:'Names', 1: 'IDs', 2: 'Prices', 3: 'Item Links', 4: 'Image Links'})
                
                files = glob.glob('CheckPoints/*')
                for f in files:
                    if f != 'CheckPoints/Checkpoint0{}.csv'.format(x-1):
                      os.remove(f)
                      print(f)
                itemz_cp.to_csv(r'CheckPoints/Checkpoint0{}.csv'.format(x), index = False)
                
                if int(x)%100==0:
                    print('Paused...')
                    time.sleep(7)
                
        return print('Done Crawling')

In [ ]:
time_now = time.time()
item_data.all_items()
time_change = time.time() - time_now




There are 20 items in this page.
There are 20 items in this page.
1
There are 20 items in this page.
There are 20 items in this page.
CheckPoints/Checkpoint0588.csv
CheckPoints/Checkpoint0589.csv
2
There are 20 items in this page.
There are 20 items in this page.
3
There are 20 items in this page.
There are 20 items in this page.
CheckPoints/Checkpoint01.csv
4
There are 20 items in this page.
There are 20 items in this page.
CheckPoints/Checkpoint02.csv
5
There are 20 items in this page.
There are 20 items in this page.
CheckPoints/Checkpoint03.csv
6
There are 20 items in this page.
There are 20 items in this page.
CheckPoints/Checkpoint04.csv
7
There are 20 items in this page.
There are 20 items in this page.
CheckPoints/Checkpoint05.csv
8
There are 20 items in this page.
There are 20 items in this page.
CheckPoints/Checkpoint06.csv
9
There are 20 items in this page.
There are 20 items in this page.
CheckPoints/Checkpoint07.csv
10
There are 20 items in this page.
There are 20 items in

In [ ]:
files = glob.glob('CheckPoints/*')
files[1]
x = int(re.sub("[^0-9]", "", files[1]))
x , time_change 

589

In [ ]:
df = pd.read_csv('CheckPoints/Checkpoint0589.csv')
n = 0
links = np.array(df['Item Links'])[n:]

y = n; dy = 1
barcodes = []
linksz = []
for i in links:
    y = y + dy
    url = i
    try:
      barcode = pd.io.html.read_html(url)[1]
      for k in range(len(barcode[0])):
          if barcode[0][k] == 'Main Barcode':
              barcodex = barcode[1][k]
              barcodes.append(barcodex)
              linksz.append((y, barcodex ))
              print( linksz[-1] )
              if y % 200 == 0:
                time.sleep(7)
    except:   pass
        
  


Streaming output truncated to the last 5000 lines.
(6745, '6001108090102')
(6746, '6003326012355')
(6747, '6001224413762')
(6748, '6009705710713')
(6749, '6003326012812')
(6750, '6001052011352')
(6751, '6009708050465')
(6752, '6003326012805')
(6753, '6009705710744')
(6754, '6009705710751')
(6755, '6009705710768')
(6756, '6009705710737')
(6757, '6009705710720')
(6758, '2595530000005')
(6759, '6009518602229')
(6760, '2595570000003')
(6761, '6001067604648')
(6762, '6001064000405')
(6763, '2595600000003')
(6764, '6001498051165')
(6765, '6001067604655')
(6766, '6001224429312')
(6767, '6001224429329')
(6768, '5900020029348')
(6769, '6009702442976')
(6770, '6009518201996')
(6771, '6009518202061')
(6772, '5449000664839')
(6773, '6009702442990')
(6774, '6009518202122')
(6775, '5449000664723')
(6776, '5449000664808')
(6777, '5449000664747')
(6778, '6005277010619')
(6779, '5449000664884')
(6780, '6001224497212')
(6781, '5449000284303')
(6782, '6009612471288')
(6783, '6009612471318')
(6784, '60012

In [ ]:
data = np.array(linksz)
pd.DataFrame(data).to_csv("CheckPoints/BarCodes.csv")